In [3]:
import numpy as np
import pandas as pd
import requests
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from pyts.image import GramianAngularField
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from imblearn.over_sampling import ADASYN, RandomOverSampler
import talib as ta
import datetime
import pickle
import time

# Parameters
window_size = 10
batch_size = 8
epochs = 200
learning_rate = 0.001

def fetch_and_preprocess_data():
    def pivotid(df1, l, n1, n2): 
        if (l-n1 < 0 or l+n2 >= len(df1)):
            return 0
        
        pividlow = 1
        pividhigh = 1
        for i in range(l-n1, l+n2+1):
            if df1.Low[l] > df1.Low[i]:
                pividlow = 0
            if df1.High[l] < df1.High[i]:
                pividhigh = 0
        if pividlow and pividhigh:
            return 3
        elif pividlow:
            return 1
        elif pividhigh:
            return 2
        else:
            return 0

    def pointpos(x):
        if x['pivot'] == 1:
            return x['Low']
        elif x['pivot'] == 2:
            return x['High']
        else:
            return 0
    """Fetch real-time BTC data and preprocess it for training."""
    end_time = int(datetime.datetime.now().timestamp() * 1000)
    start_time = int((datetime.datetime.now() - datetime.timedelta(days=365)).timestamp() * 1000)

    url = f"https://public.coindcx.com/market_data/candles?pair=I-BTC_INR&interval=1m&startTime={start_time}&endTime={end_time}"
    
    response = requests.get(url)
    data = response.json()

    btc_data = pd.DataFrame(data)
    btc_data = btc_data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'volume': 'Volume', 'close': 'Close'})

    btc_data['time'] = pd.to_datetime(btc_data['time'], unit='ms')
    btc_data[['Open', 'High', 'Low', 'Close', 'Volume']] = btc_data[['Open', 'High', 'Low', 'Close', 'Volume']].astype(float)
    # Feature Engineering
    btc_data['rsi'] = ta.RSI(btc_data['Close'], timeperiod=14)
    btc_data['macd'], btc_data['macd_signal'], btc_data['macd_hist'] = ta.MACD(btc_data['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    btc_data['upperband'], btc_data['middleband'], btc_data['lowerband'] = ta.BBANDS(btc_data['Close'], timeperiod=20)
    btc_data['ema_50'] = ta.EMA(btc_data['Close'], timeperiod=50)
    btc_data['ema_200'] = ta.EMA(btc_data['Close'], timeperiod=200)

    # Handling NaN values
    btc_data.fillna(method='ffill', inplace=True)
    btc_data.fillna(method='bfill', inplace=True)

    # Check for remaining NaNs
    if btc_data.isnull().any().any():
        print("Warning: NaN values found after preprocessing!")

    # btc_data['pivot'] = btc_data.apply(lambda x: pivotid(btc_data, x.name, 10, 10), axis=1)
    # btc_data['pointpos'] = btc_data.apply(lambda x: pointpos(x), axis=1)

    # Target variable (Close price difference)
    btc_data['close_diff'] = btc_data['Close'].shift(-1) - btc_data['Close']
    btc_data.dropna(inplace=True)

    # Normalization
    scaler = MinMaxScaler()
    features = ['Close', 'rsi', 'macd', 'upperband', 'middleband', 'lowerband', 'macd_signal', 'macd_hist', 'ema_50', 'ema_200']
    btc_data[features] = scaler.fit_transform(btc_data[features])

    with open('scaler_binance.pkl', 'wb') as f:
        pickle.dump(scaler, f)

    # Labeling
    upper_quantile = btc_data['close_diff'].quantile(0.55)
    lower_quantile = btc_data['close_diff'].quantile(0.45)
    btc_data['label'] = btc_data['close_diff'].apply(lambda x: 1 if x > upper_quantile else 2 if x < lower_quantile else 0)

    return btc_data

def create_cnn_model(input_shape):
    """Build a CNN model that matches your GAF output dimensions"""
    model = Sequential([
        Conv3D(32, (3, 3, 3), activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling3D(pool_size=(2, 2, 2)),
        Dropout(0.1),

        Conv3D(64, (3, 3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling3D(pool_size=(2, 2, 2)),
        Dropout(0.1),

        Flatten(),
        Dense(512, activation='relu'),  # Changed to match your GAF output
        Dropout(0.1),
        Dense(3, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=learning_rate), 
                 loss='sparse_categorical_crossentropy', 
                 metrics=['accuracy'])
    return model

def create_gaf_images(data, columns, window_size):
    """Generate GAF images for CNN training."""
    gaf = GramianAngularField(method='summation', image_size=window_size)
    gaf_images = []
    for i in range(window_size, len(data)):
        window_data = data.iloc[i-window_size:i][columns].values.T
        if window_data.shape[1] < window_size:
            continue
        gaf_image = gaf.fit_transform(window_data)
        print(f"GAF Image Shape: {gaf_image.shape}")  # Debug print
        gaf_images.append(gaf_image)
    return np.array(gaf_images)

def main():
    """Main function to fetch data, preprocess, train, and evaluate the model."""
    btc_data = fetch_and_preprocess_data()
    btc_data = btc_data.sample(frac=1).reset_index(drop=True)  # Shuffle the data

    # GAF Images for CNN
    columns = ['Close', 'rsi', 'macd', 'upperband', 'middleband', 'lowerband', 'macd_signal', 'macd_hist', 'ema_50', 'ema_200']
    gaf_images = create_gaf_images(btc_data, columns, window_size)

    X_gaf = gaf_images.reshape(gaf_images.shape[0], gaf_images.shape[1], gaf_images.shape[2], gaf_images.shape[3], 1)
    y_labels = btc_data['label'].values[window_size:]

    # Check class distribution
    print("Class Distribution (Before Oversampling):", np.bincount(y_labels.astype(int)))

    # Reshape X_gaf for oversampling (flatten the last 4 dimensions)
    X_gaf_flattened = X_gaf.reshape(X_gaf.shape[0], -1)

    # Try ADASYN first
    try:
        adasyn = ADASYN(random_state=42)
        X_gaf_resampled, y_gaf_resampled = adasyn.fit_resample(X_gaf_flattened, y_labels)
        print("ADASYN oversampling applied.")
    except ValueError:
        # If ADASYN fails, use RandomOverSampler as a fallback
        ros = RandomOverSampler(random_state=42)
        X_gaf_resampled, y_gaf_resampled = ros.fit_resample(X_gaf_flattened, y_labels)
        print("ADASYN failed. RandomOverSampler applied as fallback.")

    # Reshape X_gaf_resampled back to the original shape
    X_gaf_resampled = X_gaf_resampled.reshape(-1, window_size, window_size, gaf_images.shape[1], 1)

    # Check class distribution after oversampling
    print("Class Distribution (After Oversampling):", np.bincount(y_gaf_resampled.astype(int)))

    # Train-Validation Split (Stratified)
    X_train_gaf, X_val_gaf, y_train_gaf, y_val_gaf = train_test_split(X_gaf_resampled, y_gaf_resampled, test_size=0.2, random_state=42, stratify=y_gaf_resampled)

    # Calculate class weights
    class_counts = np.bincount(y_train_gaf.astype(int))
    class_weights = {i: 1.0 / count if count > 0 else 1.0 for i, count in enumerate(class_counts)}
    print("Class Weights:", class_weights)

    # Train CNN Model
    cnn_model = create_cnn_model((window_size, window_size, gaf_images.shape[1], 1))
    
    # Define callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=30, min_delta=0.001, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=20, min_lr=0.00001, verbose=1)
    
    cnn_model.fit(
        X_train_gaf, y_train_gaf,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.3,
        callbacks=[reduce_lr],
        class_weight=class_weights
    )

    # Evaluate the model on the validation set
    y_pred = cnn_model.predict(X_val_gaf)
    y_pred_classes = np.argmax(y_pred, axis=1)  # Convert probabilities to class labels

    # Generate classification report
    print("Classification Report")
    print(classification_report(y_val_gaf, y_pred_classes, target_names=['Class 0', 'Class 1', 'Class 2']))
    
    cnn_model.save('cnn_model_coindcx_optimized.h5')
    print("\nModel saved as 'cnn_model_coindcx_optimized.h5'.")
    requests.post("http://localhost:5000/update", json={"Updated": "CNN"})

    print("Training complete! CNN model saved.")

if __name__ == "__main__":
    while True:
        main()

/var/folders/tt/ksb8v1f959s87234_wq7h4j80000gn/T/ipykernel_26517/3549005329.py:74: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  btc_data.fillna(method='ffill', inplace=True)
/var/folders/tt/ksb8v1f959s87234_wq7h4j80000gn/T/ipykernel_26517/3549005329.py:75: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  btc_data.fillna(method='bfill', inplace=True)


GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image 

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.3718 - loss: 0.0167 - val_accuracy: 0.3165 - val_loss: 1.2084 - learning_rate: 0.0010
Epoch 2/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.5326 - loss: 0.0056 - val_accuracy: 0.3165 - val_loss: 2.0997 - learning_rate: 0.0010
Epoch 3/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.5967 - loss: 0.0048 - val_accuracy: 0.3165 - val_loss: 2.6136 - learning_rate: 0.0010
Epoch 4/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.7829 - loss: 0.0032 - val_accuracy: 0.3291 - val_loss: 3.1041 - learning_rate: 0.0010
Epoch 5/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.8375 - loss: 0.0024 - val_accuracy: 0.4367 - val_loss: 3.1153 - learning_rate: 0.0010
Epoch 6/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.8363 - loss: 0.0023 - val_accuracy: 0.3228 - val_loss: 2.7773 - learning_rate: 0.0010
Epoch 7/200
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.8882 - loss: 0.0017 - val_accuracy

Classification Report
              precision    recall  f1-score   support

     Class 0       0.89      0.91      0.90        44
     Class 1       0.57      0.70      0.63        44
     Class 2       0.64      0.48      0.55        44

    accuracy                           0.70       132
   macro avg       0.70      0.70      0.69       132
weighted avg       0.70      0.70      0.69       132


Model saved as 'cnn_model_coindcx_optimized.h5'.
Training complete! CNN model saved.


...

Classification Report
              precision    recall  f1-score   support

     Class 0       0.86      0.95      0.90        44
     Class 1       0.50      0.43      0.46        44
     Class 2       0.56      0.57      0.56        44

    accuracy                           0.65       132
   macro avg       0.64      0.65      0.64       132
weighted avg       0.64      0.65      0.64       132


Model saved as 'cnn_model_coindcx_optimized.h5'.
Training complete! CNN model saved.
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
G

/var/folders/tt/ksb8v1f959s87234_wq7h4j80000gn/T/ipykernel_26517/3549005329.py:74: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  btc_data.fillna(method='ffill', inplace=True)
/var/folders/tt/ksb8v1f959s87234_wq7h4j80000gn/T/ipykernel_26517/3549005329.py:75: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  btc_data.fillna(method='bfill', inplace=True)
/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image Shape: (10, 10, 10)
GAF Image 

2025-05-17 23:11:49.469299: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [0] vs. [3]
	 [[{{function_node __inference_one_step_on_data_3652864}}{{node adam/truediv_23}}]]


InvalidArgumentError: Graph execution error:

Detected at node adam/truediv_23 defined at (most recent call last):
  File "/opt/anaconda3/lib/python3.11/runpy.py", line 198, in _run_module_as_main

  File "/opt/anaconda3/lib/python3.11/runpy.py", line 88, in _run_code

  File "/opt/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/opt/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/opt/anaconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/opt/anaconda3/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/opt/anaconda3/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/opt/anaconda3/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/opt/anaconda3/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/opt/anaconda3/lib/python3.11/site-packages/ipykernel/k